In [ ]:
#########
# The purpose of this script is to do TFCE analysis on functional connectivity data and compare the results to results 
# from Conn
# Tom Possidente - January 2025
#########

In [184]:
import numpy as np
import pandas as pd
import scipy
from mne.stats import permutation_cluster_1samp_test

**Load in gPPI betas for each condition (for each subj for each ROI for each connection) from mat file produced by Conn**

In [208]:
ROI_data_fpath = '/projectnb/somerslab/tom/projects/spacetime_network/data/conn_toolbox_folder/conn_localizer_task/results/secondlevel/gPPI/AllSubjects/aA(-1).vA(1)/conn_gPPI_betadiffs.mat'
mat = scipy.io.loadmat(ROI_data_fpath)

beta_diffs = mat['beta_diffs']
ROI_names = np.squeeze(mat['ROI_names'])
adjmat = mat['adj_mat']

# convert numpy adjmat to scipy sparse matrix
adjmat_sp = scipy.sparse.coo_matrix(adjmat)

print(np.shape(beta_diffs)) # subj x ROI x ROI_connection

print(ROI_names)

(20, 26, 26)
[array(['tgPCS (L)'], dtype='<U9') array(['FO (L)'], dtype='<U6')
 array(['CO (L)'], dtype='<U6') array(['cIFSG (L)'], dtype='<U9')
 array(['cmSFG (L)'], dtype='<U9') array(['tgPCS (R)'], dtype='<U9')
 array(['FO (R)'], dtype='<U6') array(['CO (R)'], dtype='<U6')
 array(['cIFSG (R)'], dtype='<U9') array(['cmSFG (R)'], dtype='<U9')
 array(['pAud (L)'], dtype='<U8') array(['pAud (R)'], dtype='<U8')
 array(['sPCS (L)'], dtype='<U8') array(['iPCS (L)'], dtype='<U8')
 array(['midIFS (L)'], dtype='<U10') array(['sPCS (R)'], dtype='<U8')
 array(['iPCS (R)'], dtype='<U8') array(['midIFS (R)'], dtype='<U10')
 array(['pVis (L)'], dtype='<U8') array(['pVis (R)'], dtype='<U8')
 array(['aINS (L)'], dtype='<U8') array(['preSMA (L)'], dtype='<U10')
 array(['dACC (L)'], dtype='<U8') array(['aINS (R)'], dtype='<U8')
 array(['preSMA (R)'], dtype='<U10') array(['dACC (R)'], dtype='<U8')]


**Do TFCE**

In [231]:
tfce_thresholds = dict(start=0, step=0.01)
n_permutations = 1000
t_tfce, clusters, p_tfce, H0 = permutation_cluster_1samp_test(
    beta_diffs,
    n_jobs=None,
    t_power=2,
    threshold=None,
    adjacency=adjmat_sp,
    n_permutations=n_permutations,
    out_type="mask",
)

Using a threshold of 2.093024
stat_fun(H1): min=nan max=nan
Running initial clustering …
Found 36 clusters


/scratch/2396386.1.onrcc-m256/ipykernel_1140317/3644963081.py:3: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  t_tfce, clusters, p_tfce, H0 = permutation_cluster_1samp_test(
100%|███████████████████████████| Permuting : 999/999 [00:02<00:00,  335.55it/s]


In [233]:
sig_clusters = np.array(clusters)[p_tfce<0.05]

In [267]:
for cc in range(sig_clusters.shape[0]):
    cluster = sig_clusters[cc]
    inds = np.where(cluster)
    print('cluster ' + str(cc))
    for ii in range(len(inds[0])):
        print(ROI_names[inds[0][ii]][0] + ' <-> ' + ROI_names[inds[1][ii]][0] + ' ' + str(np.round(t_tfce[inds[0][ii], inds[1][ii]],2)))
    print('')
    


cluster 0
midIFS (L) <-> pVis (L) 3.17
midIFS (L) <-> pVis (R) 2.21
sPCS (R) <-> pVis (R) 2.48
iPCS (R) <-> pVis (L) 2.45
iPCS (R) <-> pVis (R) 3.02
midIFS (R) <-> pVis (L) 4.12
midIFS (R) <-> pVis (R) 5.11

cluster 1
FO (L) <-> cmSFG (L) -3.02
FO (L) <-> FO (R) -3.67
FO (L) <-> cIFSG (R) -3.44
FO (L) <-> cmSFG (R) -4.1
CO (L) <-> cIFSG (L) -2.1
CO (L) <-> cIFSG (R) -3.82

cluster 2
cIFSG (L) <-> FO (R) -2.29
cmSFG (L) <-> FO (L) -2.32
cmSFG (L) <-> tgPCS (R) -3.77
cmSFG (L) <-> FO (R) -3.24
cmSFG (L) <-> cIFSG (R) -2.59
tgPCS (R) <-> FO (L) -2.97
tgPCS (R) <-> cIFSG (R) -3.58
FO (R) <-> FO (L) -2.57
FO (R) <-> cmSFG (L) -2.49
FO (R) <-> cIFSG (R) -3.04

cluster 3
cmSFG (L) <-> pAud (L) -4.92
cmSFG (L) <-> pAud (R) -5.16
tgPCS (R) <-> pAud (L) -3.63
tgPCS (R) <-> pAud (R) -3.9
FO (R) <-> pAud (R) -2.53

cluster 4
cIFSG (R) <-> FO (L) -3.52
cIFSG (R) <-> CO (L) -2.25
cIFSG (R) <-> cmSFG (L) -2.23
cIFSG (R) <-> FO (R) -2.22
cmSFG (R) <-> FO (L) -3.78
pAud (L) <-> FO (L) -2.49
pAud (L) <-

In [269]:
p_tfce[p_tfce<0.05]

array([0.028, 0.04 , 0.018, 0.018, 0.003])